In [ ]:
%cd ~
!pwd

In [ ]:
import os 
def makedirs(output_dir):
    os.makedirs(os.path.dirname(output_dir), exist_ok=True)
    return output_dir

In [ ]:
import gzip
import numpy as np
import soundfile as sf 
import librosa
target_sr=16000

def convert_gz_to_wav(input_file, output_file):
    with gzip.open(input_file, 'rb') as f_in:
        npz_data = np.load(f_in)
        print(npz_data.files)
        audio=npz_data['audio']
        fs=npz_data['fs']
        resampled_audio=librosa.resample(audio,orig_sr=fs,target_sr=target_sr)
        sf.write(makedirs(output_file),resampled_audio,samplerate=target_sr)

# 指定输入和输出文件名
gz_files=[i for i in os.listdir('datasets/auditory_eeg_decoding/lbollens/sparrkulee/stimuli/eeg/') if i[-7:]=='.npz.gz']
for input_file in gz_files:
    if input_file.startswith(['audiobook','podcast']):
        if 'mel' in input_file:
            continue
        if input_file.split('.')[0][-1].isdigit():
            input_file = f'datasets/auditory_eeg_decoding/lbollens/sparrkulee/stimuli/eeg/{input_file}'
            output_file = input_file.replace('npz.gz','wav').replace('stimuli','processed_audio')
            print(output_file)
            convert_gz_to_wav(input_file,output_file)
# with gzip.open(input_file, 'rb') as f_in:
#     npz_data = np.load(f_in)
#     print(npz_data['audio'].shape)
# 解压缩 .npz.gz 文件为 .npz 文件
# extract_npz_gz(input_file, output_file)

In [ ]:
# 开始转录
import whisper
import tqdm
import json
model = whisper.load_model('large')
wav_files=[i for i in os.listdir(output_dir) if i[-4:]=='.wav']
# 从音频转写到文本。还需要把音频转为16kHz，写入json文件
for wav_path in tqdm.tqdm(wav_files[:2]):
    result = model.transcribe(
        wav_path, language="Dutch", word_timestamps=True,
        without_timestamps=False)
    

In [ ]:
input_file='datasets/auditory_eeg_decoding/lbollens/sparrkulee/stimuli/eeg/audiobook_5_1.npz.gz'
output_file='datasets/auditory_eeg_decoding/lbollens/sparrkulee/stimuli/processed_audio/tmp.wav'
with gzip.open(input_file, 'rb') as f_in:
    npz_data = np.load(f_in)
    print(npz_data.files)
    audio=npz_data['audio']
    fs=npz_data['fs']
    resampled_audio=librosa.resample(audio,orig_sr=fs,target_sr=target_sr)
    sf.write(makedirs(output_file),resampled_audio,samplerate=target_sr)
from IPython.display import Audio,display
display(Audio(output_file))

In [ ]:
!jupyter notebook stop


In [ ]:
15*60+6

In [ ]:
import torch
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"

pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-base",
  chunk_length_s=30,
  device=device
)
# huggingface-cli download --resume-download openai/whisper-large-v3 --local-dir ~/transformers_models/openai-whisper-large-v3

In [ ]:
from IPython.display import Audio,display
display(Audio('datasets/auditory_eeg_decoding/lbollens/sparrkulee/processed_audio/eeg/audiobook_13_2.wav'))

In [ ]:

from transformers import WhisperProcessor

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="dutch", task="transcribe")
sample,sr = sf.read('datasets/auditory_eeg_decoding/lbollens/sparrkulee/processed_audio/eeg/audiobook_13_2.wav')
generate_kwargs= {
    "max_length":20,
    "do_sample":False,
    "num_beams":5,
    "repetition_penalty" :5.0,
    "forced_decoder_ids":forced_decoder_ids
}
# we can also return timestamps for the predictions
prediction = pipe(sample, batch_size=8, return_timestamps=True,
                  generate_kwargs=generate_kwargs
                  )["chunks"]
print(prediction)

In [ ]:
# 开始转录
import whisper
import tqdm
import json
model = whisper.load_model('large')
wav_files=[i for i in os.listdir('datasets/auditory_eeg_decoding/lbollens/sparrkulee/processed_audio/eeg') if i[-4:]=='.wav']
# 从音频转写到文本。还需要把音频转为16kHz，写入json文件
for wav_path in tqdm.tqdm(wav_files[:2]):
    result = model.transcribe(
        wav_path, language="Dutch", word_timestamps=True,
        without_timestamps=False)

    # 写入文本
    transcribe_path = f"datasets/auditory_eeg_decoding/lbollens/sparrkulee/processed_audio/transcribe/{os.path.basename(wav_path)[:-4]}.json"
    transcribe_path = makedirs(transcribe_path)
    with open(transcribe_path, 'w') as write_f:
        json.dump(result, write_f, indent=4, ensure_ascii=False)

In [ ]:

# 原始格式是.npz.gz,我们现在先把这个音频文件变为npz，再变到16kHz wav,方便whisper处理。
a=np.load('datasets/auditory_eeg_decoding/lbollens/sparrkulee/stimuli/eeg/audiobook_1_1_shifted.npz')
print(a.files)
print(a['audio'].shape,a['fs'])